In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

from PhaseFunctionedNetwork import PhaseFunctionedNetwork
from train_utils import train_pfnn

# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

In [2]:
# Process Data (TODO IMPROVE SO IT CAN BE LOADED INTO A SINGLE FILE)

# Idle Data
X_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Idle_Static.txt'))
Y_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Idle_Static.txt'))
P_idle = np.linspace(0, 2*np.pi, num=72)
delta_idle =  P_idle[1:] - P_idle[:-1]
delta_idle[delta_idle < 0] = (1.0 - P_idle[:-1] + P_idle[1:])[delta_idle < 0]
delta_idle = np.append(delta_idle, 0) #TODO IF BREAKS append 2pi
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}, {delta_idle.shape}")

# Jump Flat
X_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizFlat.txt'))
Y_jump_horizflat = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizFlat.txt'))
P_jump_horizflat = np.linspace(0, 2*np.pi, num=50)
delta_jump_horizflat =  P_jump_horizflat[1:] - P_jump_horizflat[:-1]
delta_jump_horizflat[delta_jump_horizflat < 0] = (1.0 - P_jump_horizflat[:-1] + P_jump_horizflat[1:])[delta_jump_horizflat < 0]
delta_jump_horizflat = np.append(delta_jump_horizflat, 0) #TODO IF BREAKS append 2pi
print(f"Horizflat data shape: {X_jump_horizflat.shape}, {Y_jump_horizflat.shape}, {P_jump_horizflat.shape}, {delta_jump_horizflat.shape}")

# Jump Up
X_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizUp.txt'))
Y_jump_horizup = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizUp.txt'))
P_jump_horizup = np.linspace(0, 2*np.pi, num=48)
delta_jump_horizup =  P_jump_horizup[1:] - P_jump_horizup[:-1]
delta_jump_horizup[delta_jump_horizup < 0] = (1.0 - P_jump_horizup[:-1] + P_jump_horizup[1:])[delta_jump_horizup < 0]
delta_jump_horizup = np.append(delta_jump_horizup, 0) #TODO IF BREAKS append 2pi
print(f"Horizup data shape: {X_jump_horizup.shape}, {Y_jump_horizup.shape}, {P_jump_horizup.shape}, {delta_jump_horizup.shape}")

# Jump Down
X_jump_horizdown = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Jump_HorizDown.txt'))
Y_jump_horizdown = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Jump_HorizDown.txt'))
P_jump_horizdown = np.linspace(0, 2*np.pi, num=56)
delta_jump_horizdown =  P_jump_horizdown[1:] - P_jump_horizdown[:-1]
delta_jump_horizdown[delta_jump_horizdown < 0] = (1.0 - P_jump_horizdown[:-1] + P_jump_horizdown[1:])[delta_jump_horizdown < 0]
delta_jump_horizdown = np.append(delta_jump_horizdown, 0) #TODO IF BREAKS append 2pi
print(f"Horizdown data shape: {X_jump_horizdown.shape}, {Y_jump_horizdown.shape}, {P_jump_horizdown.shape}, {delta_jump_horizdown.shape}")

# Idle Looking
X_idle_looking = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Idle_Looking.txt'))
Y_idle_looking = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Idle_Looking.txt'))
P_idle_looking = np.linspace(0, 2*np.pi, num=1192)
delta_idle_looking =  P_idle_looking[1:] - P_idle_looking[:-1]
delta_idle_looking[delta_idle_looking < 0] = (1.0 - P_idle_looking[:-1] + P_idle_looking[1:])[delta_idle_looking < 0]
delta_idle_looking = np.append(delta_idle_looking, 0) #TODO IF BREAKS append 2pi
print(f"Idle looking data shape: {X_idle_looking.shape}, {Y_idle_looking.shape}, {P_idle_looking.shape}, {delta_idle_looking.shape}")

Idle data shape: (72, 1836), (72, 1767), (72,), (72,)
Horizflat data shape: (50, 1836), (50, 1767), (50,), (50,)
Horizup data shape: (48, 1836), (48, 1767), (48,), (48,)
Horizdown data shape: (56, 1836), (56, 1767), (56,), (56,)
Idle looking data shape: (1192, 1836), (1192, 1767), (1192,), (1192,)


In [4]:
# WEIGH THE JOINT (in particular all the hand and foot joints need to be weighted less)
JOINT_NUM = 144

joint_weights = np.array([
    1,                                                  # HIPS
    1, 1, 1,                                            # LEFT leg
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot thumb
    1,                                                  # LEFT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # LEFT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT foot ring
    1, 1, 1,                                            # RIGHT leg
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot thumb
    1,                                                  # RIGHT foot toe base
    1e-10, 1e-10, 1e-10, 1,                             # RIGHT foot index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT foot ring
    1, 1, 1, 1,                                         # SPINE
    1, 1,                                               # LEFT shoulder
    1, 1,                                               # LEFT arm
    1,                                                  # LEFT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # LEFT hand ring
    1, 1e-10, 1,                                        # LEFT wing feathers large
    1, 1e-10, 1,                                        # LEFT wing feathers medium
    1,                                                  # LEFT wing feathers small
    1, 1, 1, 1, 1, 1, 1,                                # NECK
    1,                                                  # HEAD
    1e-10, 1e-10,                                       # JAW
    1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10, 1e-10,    # TONUGE
    1e-10, 1e-10, 1e-10,                                # LEFT eye
    1e-10, 1e-10, 1e-10,                                # RIGHT eye
    1, 1,                                               # RIGHT shoulder
    1, 1,                                               # RIGHT arm
    1,                                                  # RIGHT hand
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand index
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand middle
    1e-10, 1e-10, 1e-10, 1e-10, 1,                      # RIGHT hand ring
    1, 1e-10, 1,                                        # RIGHT wing feathers large
    1, 1e-10, 1,                                        # RIGHT wing feathers medium
    1,                                                  # RIGHT wing feathers small
    1, 1, 1, 1, 1, 1, 1, 1, 1,                          # TAIL
    1e-10, 1e-10, 1,                                    # LEFT tail feather
    1e-10, 1e-10, 1,                                    # MIDDLE tail feather         
    1e-10, 1e-10, 1                                     # RIGHT tail feather 
])

# repeat weights for each joint to represent X, Y and Z
joint_weights = joint_weights.repeat(3)

In [5]:
def preprocess_X(X_arr):
    Xmean, Xstd = X_arr.mean(axis=0), X_arr.std(axis=0)

    # lists to keep track of indices for TRAJECTORY
    X_traj_pos_indices = []
    X_traj_dir_indices = []
    X_traj_vel_indices = []
    X_traj_speed_indices = []
    X_traj_style_indices = []

    # number of eleements for each trajectory point
    w = 9
    for i in range(0, 107, w):
        X_traj_pos_indices = np.append(X_traj_pos_indices, range(i,i+2)).astype(int)
        X_traj_dir_indices = np.append(X_traj_dir_indices, range(i+2,i+4)).astype(int)
        X_traj_vel_indices = np.append(X_traj_vel_indices, range(i+4,i+6)).astype(int)
        X_traj_speed_indices = np.append(X_traj_speed_indices, i+6).astype(int)
        X_traj_style_indices = np.append(X_traj_style_indices, range(i+7,i+9)).astype(int)

    # lists to keep track of indices for JOINTS
    X_joint_pos_indices = []
    X_joint_for_indices = []
    X_joint_up_indices = []
    X_joint_vel_indices = []

    # num of elements for each joint
    w = 12
    for i in range(108, 1835, w):
        X_joint_pos_indices = np.append(X_joint_pos_indices, range(i,i+3)).astype(int)
        X_joint_for_indices = np.append(X_joint_for_indices, range(i+3,i+6)).astype(int)
        X_joint_up_indices = np.append(X_joint_up_indices, range(i+6,i+9)).astype(int)
        X_joint_vel_indices = np.append(X_joint_vel_indices, range(i+9,i+12)).astype(int)

    # INPUT Trajectory data
    Xstd[X_traj_pos_indices] = Xstd[X_traj_pos_indices].mean()
    Xstd[X_traj_dir_indices] = Xstd[X_traj_dir_indices].mean()
    Xstd[X_traj_vel_indices] = Xstd[X_traj_vel_indices].mean()
    Xstd[X_traj_speed_indices] = Xstd[X_traj_speed_indices].mean()
    Xstd[X_traj_style_indices] = Xstd[X_traj_style_indices].mean()

    # INPUT Joint data --> This is where we weight the joints
    # Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_for_indices] = Xstd[X_joint_for_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_up_indices] = Xstd[X_joint_up_indices].mean() / (joint_weights * 0.1)
    # Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean() / (joint_weights * 0.1)
    Xstd[X_joint_pos_indices] = Xstd[X_joint_pos_indices].mean() 
    Xstd[X_joint_for_indices] = Xstd[X_joint_for_indices].mean()
    Xstd[X_joint_up_indices] = Xstd[X_joint_up_indices].mean()
    Xstd[X_joint_vel_indices] = Xstd[X_joint_vel_indices].mean()

    return Xmean, Xstd

def preprocess_Y(Y_arr):
    Ymean, Ystd = Y_arr.mean(axis=0), Y_arr.std(axis=0)

    # PREPROCESS OUTPUT Y
    # lists to keep track of indices for TRAJECTORY
    Y_traj_pos_indices = []
    Y_traj_dir_indices = []
    Y_traj_vel_indices = []

    # number of trajectory elements
    w = 6
    for i in range(0, 35, w):
        Y_traj_pos_indices = np.append(Y_traj_pos_indices, range(i,i+2)).astype(int)
        Y_traj_dir_indices = np.append(Y_traj_dir_indices, range(i+2,i+4)).astype(int)
        Y_traj_vel_indices = np.append(Y_traj_vel_indices, range(i+4,i+6)).astype(int)

    # lists to keep track of indices for JOINTS
    Y_joint_pos_indices = []
    Y_joint_for_indices = []
    Y_joint_up_indices = []
    Y_joint_vel_indices = []

    # num of joint elements
    w = 12
    for i in range(36, 1763, w):
        Y_joint_pos_indices = np.append(Y_joint_pos_indices, range(i,i+3)).astype(int)
        Y_joint_for_indices = np.append(Y_joint_for_indices, range(i+3,i+6)).astype(int)
        Y_joint_up_indices = np.append(Y_joint_up_indices, range(i+6,i+9)).astype(int)
        Y_joint_vel_indices = np.append(Y_joint_vel_indices, range(i+9,i+12)).astype(int)

    # OUTPUT Trajectory data
    Ystd[Y_traj_pos_indices] = Ystd[Y_traj_pos_indices].mean()
    Ystd[Y_traj_dir_indices] = Ystd[Y_traj_dir_indices].mean()
    Ystd[Y_traj_vel_indices] = Ystd[Y_traj_vel_indices].mean()

    # OUTPUT Joint data --> This is where we weight the joints
    # Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_for_indices] = Ystd[Y_joint_for_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_up_indices] = Ystd[Y_joint_up_indices].mean() / (joint_weights * 0.1)
    # Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean() / (joint_weights * 0.1)
    Ystd[Y_joint_pos_indices] = Ystd[Y_joint_pos_indices].mean()
    Ystd[Y_joint_for_indices] = Ystd[Y_joint_for_indices].mean()
    Ystd[Y_joint_up_indices] = Ystd[Y_joint_up_indices].mean() 
    Ystd[Y_joint_vel_indices] = Ystd[Y_joint_vel_indices].mean()
    
    #TODO MAYBE ROOT TRASLATION arr[-3:] --> last three elements

    return Ymean, Ystd

In [6]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_idle, X_jump_horizflat, X_jump_horizup, X_jump_horizdown))
Y = np.concatenate((Y_idle, Y_jump_horizflat, Y_jump_horizup, Y_jump_horizdown))
P = np.concatenate((P_idle, P_jump_horizflat, P_jump_horizup, P_jump_horizdown))
delta_phase = np.concatenate((delta_idle, delta_jump_horizflat, delta_jump_horizup, delta_jump_horizdown))
Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

# Preprocess Data
Xmean, Xstd = preprocess_X(X)
Ymean, Ystd = preprocess_Y(Y)

WEIGHTS_SAVE_PATH = 'C:/Users/Ana/Desktop/dev/pfnn-dev/unity-pfnn/Assets/Dev/Weights/test/'

# save means and stds
Xmean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xmean.bin')
Ymean.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ymean.bin')
Xstd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Xstd.bin')
Ystd.astype(np.float32).tofile(WEIGHTS_SAVE_PATH + 'Ystd.bin')

# normalize data NOTE ORIGINAL DID THIS AFTER SAVING THE MEANS AND STD AS DONE HERE
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

# load data for PyTorch training

# append phase as additional feature only for training NN
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

Input shape torch.Size([226, 1837])
Target shape torch.Size([226, 1768])


In [21]:
# Define PFNN
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1])

# Determine device for training 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)

# Training variables
BATCH_SIZE = 32
EPOCHS = 4
LR = 0.001
OPTIMIZER = torch.optim.AdamW(model.parameters(), lr=LR)
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Train
model = train_pfnn(model, train_dataloader, optimizer=OPTIMIZER, num_epochs=EPOCHS, device=DEVICE)

# Save
model.precompute_and_save_weights()

100%|██████████| 8/8 [00:01<00:00,  5.53it/s]


Epoch [1/4], Loss: 1.6373355102217997


100%|██████████| 8/8 [00:01<00:00,  7.04it/s]


Epoch [2/4], Loss: 1.0678104138364073


100%|██████████| 8/8 [00:01<00:00,  7.00it/s]


Epoch [3/4], Loss: 0.8098547694280871


100%|██████████| 8/8 [00:01<00:00,  7.00it/s]


Epoch [4/4], Loss: 0.8293232762021514
